## Web Assembly

In [1]:
#include <base/net/HTTPSRequest.h>
#include <base/string/Format.h>
#include <base/string/FormatOutputStream.h>
#include <base/webassembly/WebAssembly.h>

In [2]:
using namespace base;

In [3]:
String url = "https://github.com/rene-fonseca/docker-webassembly/raw/master/wasmtime/hello";

In [4]:
auto response = HTTPSRequest::load(url)

In [5]:
response.getSecond()

"application/octet-stream"

In [6]:
response.getFirst().getLength()

31259

In [7]:
WebAssembly::isSupported()

true

In [8]:
WebAssembly wasm;

In [9]:
wasm.getEngine()

"Wasmtime 0.12"

In [10]:
wasm.isValid(response.getFirst())

true

In [11]:
wasm.load(response.getFirst())

true

In [12]:
fout << "IMPORTS:" << EOL;
for (const auto& s : wasm.getImports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY " << s.name << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

IMPORTS:
  [0] FUNCTION [i32] wasi_unstable!fd_prestat_get(i32, i32)
  [1] FUNCTION [i32] wasi_unstable!fd_prestat_dir_name(i32, i32, i32)
  [2] FUNCTION [] wasi_unstable!proc_exit(i32)
  [3] FUNCTION [i32] wasi_unstable!fd_fdstat_get(i32, i32)
  [4] FUNCTION [i32] wasi_unstable!fd_close(i32)
  [5] FUNCTION [i32] wasi_unstable!args_sizes_get(i32, i32)
  [6] FUNCTION [i32] wasi_unstable!args_get(i32, i32)
  [7] FUNCTION [i32] wasi_unstable!fd_write(i32, i32, i32, i32)
  [8] FUNCTION [i32] wasi_unstable!fd_seek(i32, i64, i32, i32)


In [13]:
wasm.makeWASIInstance(nullptr, nullptr, nullptr)

true

In [14]:
fout << "EXPORTS:" << ENDL;
for (const auto& s : wasm.getExports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY SIZE=" << s.memorySize << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

EXPORTS:


In [15]:
// wasm.call("_start")

In [16]:
const char* wat = R"WAT((module
    ;; Import the required fd_write WASI function which will write the given io vectors to stdout
    ;; The function signature for fd_write is:
    ;; (File Descriptor, *iovs, iovs_len, nwritten) -> Returns number of bytes written
    (import "wasi_unstable" "fd_write" (func $fd_write (param i32 i32 i32 i32) (result i32)))

    (memory 1)
    (export "memory" (memory 0))

    ;; Write 'hello world\n' to memory at an offset of 8 bytes
    ;; Note the trailing newline which is required for the text to appear
    (data (i32.const 8) "hello world\n")

    (func $main (export "_start")
        ;; Creating a new io vector within linear memory
        (i32.store (i32.const 0) (i32.const 8))  ;; iov.iov_base - This is a pointer to the start of the 'hello world\n' string
        (i32.store (i32.const 4) (i32.const 12))  ;; iov.iov_len - The length of the 'hello world\n' string

        (call $fd_write
            (i32.const 1) ;; file_descriptor - 1 for stdout
            (i32.const 0) ;; *iovs - The pointer to the iov array, which is stored at memory location 0
            (i32.const 1) ;; iovs_len - We're printing 1 string stored in an iov - so one.
            (i32.const 20) ;; nwritten - A place in memory to store the number of bytes written
        )
        drop ;; Discard the number of bytes written from the top of the stack
    )
))WAT";

In [17]:
WebAssembly wasm2;

Convert Text format (WAT) to WASM.

In [18]:
try {
  String wasm = WebAssembly::convertWATToWASM(wat);
  fout << "WASM size: " << wasm.getLength() << ENDL;
} catch (WebAssembly::WebAssemblyException& e) {
  ferr << e << ENDL;
}

WASM size: 157


We can load either WAT and WASM with loadAny() so we do not have to convert WAT To WASM explicitly.

In [19]:
wasm2.loadAny(wat)

true

In [20]:
wasm2.makeInstance(true)

true

In [21]:
fout << "EXPORTS:" << ENDL;
for (const auto& s : wasm2.getExports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY SIZE=" << s.memorySize << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

EXPORTS:
  [0] MEMORY SIZE=65536
  [1] FUNCTION [] _start()


In [22]:
wasm2.call("_start")

 [TYPE=void]

In [23]:
const char* addWAT = R"WAT((module
  (type (;0;) (func (param i32 i32) (result i32)))
  (func $add (type 0) (param i32 i32) (result i32)
    get_local 1
    get_local 0
    i32.add)
  (memory 1)
  (export "memory" (memory 0))
  (export "add" (func $add))
))WAT";

In [24]:
WebAssembly wasm3;
wasm3.loadAny(addWAT) && wasm3.makeInstance()

true

In [25]:
fout << "EXPORTS:" << ENDL;
for (const auto& s : wasm3.getExports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY SIZE=" << s.memorySize << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

EXPORTS:
  [0] MEMORY SIZE=65536
  [1] FUNCTION [i32] add(i32, i32)


In [26]:
wasm3.getImports()

base::Array [SIZE=0] Index Value

In [27]:
wasm3.getExports()

base::Array [SIZE=2] Index Value 0 MEMORY SIZE=65536 1 [i32] add(i32, i32)

In [28]:
wasm3.invoke<int>("add", -1234, 4567)

3333